In [1]:
import pickle
import pandas as pd

# Data Exploration

## Look and Loaded (load data and look at it)

In [30]:
def load_pickle(file):
    objects = []
    with open(file, "rb") as openfile:
        u = pickle._Unpickler(openfile)
        u.encoding = 'latin1'
        df = u.load()
    return df

In [18]:
wthr_df = load_pickle("griddata_gfs_us_20180101_20190826_03_final.p")

In [36]:
wthr_df.head()

,dswrf_sfc_wm2,gust_sfc_ms,hpbl_sfc_m,msl_ms_pa,pottemp_80mto0m_K,q_2m_1,q_pfg30_1,r_pl925_%,r_pl950_%,shtfl_sfc_wm2,...,wdir_10m_dn,wdir_pfg30_dn,wdir_pl925_dn,wsp_100m_ms,wsp_10m_ms,wsp_pfg30_ms,wsp_pl925_ms,created_on,azimuth,elevation
index,,,,,,,,,,,,,,,,,,,,,
2018-01-01 06:00:00+00:00,0.0,16.777032,1349.927656,99212.0625,0.559356,0.005105,0.004852,89.975,83.075,-58.444885,...,51.936818,54.855777,58.963042,16.548291,9.772748,16.622077,21.963719,2018-01-01 00:00:00+00:00,111.880959,-12.132894
2018-01-01 07:00:00+00:00,0.0,16.606744,1352.623506,99259.8125,0.528755,0.005005,0.004739,89.250,82.250,-59.127808,...,59.839207,62.809819,66.607932,16.795002,9.966123,16.832585,21.917341,2018-01-01 00:00:00+00:00,122.901711,-3.799353
2018-01-01 08:00:00+00:00,0.0,16.750567,1385.881213,99340.0125,0.519516,0.004875,0.004601,88.350,81.475,-60.174194,...,66.846392,69.702886,73.395684,17.187174,10.191553,17.103298,22.151705,2018-01-01 00:00:00+00:00,134.513901,3.528108
2018-01-01 09:00:00+00:00,30.0,16.653720,1406.317913,99435.6875,0.462775,0.004693,0.004395,85.400,78.575,-61.376221,...,69.538877,72.116654,75.992542,16.804585,10.074198,16.507619,21.857454,2018-01-01 00:00:00+00:00,146.969666,9.473817
2018-01-01 10:00:00+00:00,70.0,16.143047,1413.328074,99523.6750,0.395166,0.004618,0.004322,85.275,78.300,-50.711792,...,67.391618,69.753380,73.848494,15.970588,9.719457,15.561127,21.082533,2018-01-01 00:00:00+00:00,160.319397,13.648098


In [40]:
max(wthr_df.index)

Timestamp('2019-08-26 18:00:00+0000', tz='UTC')

In [20]:
wthr_df.shape

(14378, 26)

In [22]:
wthr_df.describe()

,dswrf_sfc_wm2,gust_sfc_ms,hpbl_sfc_m,msl_ms_pa,pottemp_80mto0m_K,q_2m_1,q_pfg30_1,r_pl925_%,r_pl950_%,shtfl_sfc_wm2,...,wdir_100m_dn,wdir_10m_dn,wdir_pfg30_dn,wdir_pl925_dn,wsp_100m_ms,wsp_10m_ms,wsp_pfg30_ms,wsp_pl925_ms,azimuth,elevation
count,14378.000000,14378.000000,14378.000000,14378.000000,14378.000000,14378.000000,14378.000000,14378.000000,14378.000000,14378.000000,...,14378.000000,14378.000000,14378.000000,14378.000000,14378.000000,14378.000000,14378.000000,14378.000000,14378.000000,14378.000000
mean,155.029194,7.943250,549.354510,101512.216593,0.742688,0.006481,0.006165,75.716828,75.737091,6.316193,...,162.410028,162.399371,162.002774,154.312530,7.471580,4.409038,7.351590,10.082908,182.032028,2.355874
std,230.147722,4.061036,427.125419,996.863087,1.175120,0.002735,0.002540,18.950255,17.724822,71.012102,...,99.361734,99.942995,99.257628,98.208108,3.329532,2.078577,3.285841,5.270483,103.042725,30.472445
min,0.000000,0.517106,21.333019,97637.887500,-2.431875,0.000993,0.000939,2.400000,2.425000,-115.227905,...,1.065702,1.605544,0.878477,1.231885,0.558441,0.389727,0.515088,0.338407,8.330822,-62.201588
25%,0.000000,4.683080,199.087539,100968.224375,-0.136376,0.004248,0.004075,64.625000,64.718129,-36.246123,...,78.421911,77.080156,78.444561,75.816115,5.176560,2.884135,5.082444,6.094260,90.158264,-19.496047
50%,15.000000,7.678551,451.845422,101567.477500,0.631883,0.006143,0.005920,79.475000,78.600000,-20.870140,...,130.455007,129.200547,130.106612,123.520460,7.159732,4.032932,7.046303,9.377695,186.476692,2.363771
75%,248.750000,10.631585,821.578707,102143.115625,1.514157,0.008523,0.008049,90.793750,89.643750,24.313370,...,261.226496,259.620900,261.262503,255.757132,9.261023,5.634826,9.126081,13.150352,273.261543,24.920794
max,885.500000,24.316044,2519.267852,104261.237500,4.514006,0.016743,0.014403,100.000000,100.000000,372.421576,...,359.180650,358.364902,359.093417,359.693548,23.256653,14.571061,23.972086,30.798095,358.682526,62.103596


In [24]:
nrg_df = load_pickle("obs_20180101_20190625_03_final_normed.p")

In [50]:
nrg_df.head()

,power_mw_obsnorm,power_available_mw_obsnorm,target_losses_norm
2018-01-01 00:00:00+01:00,0.330269,0.450934,0.120665
2018-01-01 00:10:00+01:00,0.339204,0.457539,0.118335
2018-01-01 00:20:00+01:00,0.336993,0.456577,0.119584
2018-01-01 00:30:00+01:00,0.337858,0.462447,0.124589
2018-01-01 00:40:00+01:00,0.353646,0.483538,0.129892


In [39]:
max(nrg_df.index)

Timestamp('2019-06-25 01:50:00+0200', tz='Europe/Berlin')

In [28]:
nrg_df.shape

(70710, 3)

In [29]:
nrg_df.describe()

,power_mw_obsnorm,power_available_mw_obsnorm,target_losses_norm
count,70710.000000,70710.000000,70710.000000
mean,0.204310,0.274691,0.070381
std,0.162364,0.264172,0.128426
min,0.000000,0.000073,0.000049
25%,0.065503,0.074167,0.005369
50%,0.166234,0.181165,0.014100
75%,0.320650,0.391951,0.056041
max,0.801717,1.019565,0.779721


## Let's get plottin'

In [57]:
nrg_df.index[14].tzinfo

<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>

In [86]:
nrg_df['date_by_hour'] = 'NaN'
for i, row in nrg_df.iterrows():
    nrg_df.date_by_hour[i] = pd.datetime(i.year, 
                                      i.month, 
                                      i.day, 
                                      i.hour, 
                                      tzinfo = i.tzinfo)

C:\Users\thoma\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\thoma\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [87]:
nrg_df.date_by_hour.head(12)

2018-01-01 00:00:00+01:00    2018-01-01 00:00:00+01:00
2018-01-01 00:10:00+01:00    2018-01-01 00:00:00+01:00
2018-01-01 00:20:00+01:00    2018-01-01 00:00:00+01:00
2018-01-01 00:30:00+01:00    2018-01-01 00:00:00+01:00
2018-01-01 00:40:00+01:00    2018-01-01 00:00:00+01:00
2018-01-01 00:50:00+01:00    2018-01-01 00:00:00+01:00
2018-01-01 01:00:00+01:00    2018-01-01 01:00:00+01:00
2018-01-01 01:10:00+01:00    2018-01-01 01:00:00+01:00
2018-01-01 01:20:00+01:00    2018-01-01 01:00:00+01:00
2018-01-01 01:30:00+01:00    2018-01-01 01:00:00+01:00
2018-01-01 01:40:00+01:00    2018-01-01 01:00:00+01:00
2018-01-01 01:50:00+01:00    2018-01-01 01:00:00+01:00
Name: date_by_hour, dtype: object

In [66]:
pd.Timestamp(2019, 2, 15)

Timestamp('2019-02-15 00:00:00')